In [16]:
import json
import os
import re
import pickle
from tqdm import tqdm 
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from collections import defaultdict
from dotenv import load_dotenv

from openai import OpenAI
import google.generativeai as genai
import anthropic
from utils import run_gpt#, run_claude

from langchain_community.utilities import SearchApiAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.tools import Tool, DuckDuckGoSearchResults


In [ ]:
from utils import run_local_llm
from ollama import Client
import json

#local llm
web_use = True
navi_use = True
llm = "llama3-gradient:70b"
client = Client(host="http://localhost:11434")

initial_question = """You are an AI agent with access to internet search and web navigation tools.

TASK: Find a Cornell University computer science professor's name, email, and research area.

AVAILABLE TOOLS:
1. search_internet - Search the internet for information
2. navigate_website - Navigate to a website and fetch content  
3. get_clickable_elements - Find clickable elements on a webpage

EXACT TOOL USAGE FORMAT:
For search_internet: {"tool": "search_internet", "search_term": "your search term"}
For navigate_website: {"tool": "navigate_website", "url": "website url"}
For get_clickable_elements: {"tool": "get_clickable_elements", "url": "website url"}

STEP-BY-STEP INSTRUCTIONS:
1. First, search for "Cornell University computer science department faculty"
2. Navigate to the faculty page URL you find
3. Extract professor information from the page
4. Format your final answer exactly as shown below

REQUIRED OUTPUT FORMAT:
Professor: [Professor Name]
Email: [Email Address]
Research Area: [Research Area]

Start now by searching for Cornell University computer science department faculty."""
subsequent_questions = []

print("=== 디버깅 시작 ===")
print(f"초기 질문: {initial_question}")
print("=" * 50)

final_response, final_responses, messages = run_local_llm(client, initial_question, subsequent_questions, llm=llm, web=web_use, navigation=navi_use, T=0.7)

print("\n=== 최종 결과 ===")
print(f"최종 응답: {final_response}")
print("\n=== 모든 응답들 ===")
for i, response in enumerate(final_responses, 1):
    print(f"{i}. {response[:200]}{'...' if len(response) > 200 else ''}")
print("\n=== 메시지 히스토리 ===")
for i, msg in enumerate(messages, 1):
    role = msg.get('role', 'unknown')
    content = msg.get('content', '')[:100] + '...' if len(str(msg.get('content', ''))) > 100 else str(msg.get('content', ''))
    print(f"{i}. [{role}] {content}")

In [23]:
###GPT, SearchAgent###
load_dotenv()
question = "Search all Postech CSE professor's name, email and research area information. You must search all information."
sub_question = []
web_use = True
navi_use = True
llm = "gpt-4o-2024-05-13"
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = api_key)
result, response, msg = run_gpt(client, question, sub_question, llm, web = web_use, navigation = navi_use)

In [24]:
print("=== GPT API 테스트 결과 ===")
print(f"Result: {result}")
print(f"Response: {response}")
print(f"Messages: {msg}")

=== GPT API 테스트 결과 ===
Result: I successfully retrieved information from the Postech CSE faculty page at [ecse.postech.ac.kr](https://ecse.postech.ac.kr/member/professor/). Here is the relevant information on professors including their emails, and research areas:

1. **Sungahn Ko**
   - Email: Not provided
   - Research Areas: Visualization, Applied AI, Social Computing, Human-Computer Interaction
   - Lab: Human-AI Interaction and Visualization Lab
   - Phone: +82-54-279-2249
   - Office: B4 121

2. **Hee-Kap Ahn**
   - Email: Not provided
   - Research Areas: Algorithms, Computational and Discrete Geometry, Data Structures, Theory of Computation
   - Lab: Algorithms Lab
   - Phone: +82-54-279-2387
   - Office: B2 233

3. **Kyungmin Bae**
   - Email: Not provided
   - Research Areas: Software Verification, Automated Reasoning, Computational Logic, Formal Methods, Software Engineering
   - Lab: Software Verification Lab
   - Phone: +82-54-279-2256
   - Office: B2 225

4. **Seung-Hwan B